In [1]:
# default_exp visualize.bokeh

# Visualize Bokeh

> Use bokeh to do visualize

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
from dataer_tool.imports import *
from bokeh.palettes import Spectral, Category20, Viridis256, Cividis256, Turbo256, Set2, brewer, Greys256, Inferno256, Magma256, Plasma256
from bokeh.io import output_file, show
from bokeh.palettes import Spectral6, Spectral
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange, LabelSet, RangeTool, HoverTool, Label, Legend, Paragraph
from bokeh.transform import cumsum, factor_cmap
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.layouts import column
from bokeh.plotting.figure import get_range, get_scale
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn


BOKEH_DEFAULT_TOOLS = 'save,reset,pan'
BOKEH_DEFAULT_TOOLBAR_LOC = 'above' #'above', 'below', 'left', 'right', None


In [4]:
#export
def count2colors(count: int):
    """
    colors from `bokeh.palettes`
    range:
        1:12    -> Spectral, 
        12:21   -> Category20, 
        21:257  -> Viridis256, 
        257:769 -> Turbo256+Cividis256+Viridis256
    """
    if count <= 0:
        raise ValueError(f"Invalid count={count}, must > 0")
    elif count <= 3:
        return Set2[3][:count]
    elif count <= 8:
        return Set2[count]
    elif count <= 20:
        return Category20[count]
    elif count <= 256:
        index = np.arange(256)
        cmp_256 = random.choice([Viridis256, Cividis256, Turbo256, Inferno256, Magma256, Plasma256])
        return tuple(np.array(cmp_256)[np.isin(index, np.arange(0, 256, 256/count, dtype=np.uint8))])
    elif count <= 256 * 3:
        return tuple(np.random.choice(Turbo256 + Cividis256 + Viridis256, count))
    else:
        raise ValueError(f"Invalid count={count}, must <= 768")

In [5]:
try:
    count2colors(-1)
except ValueError as e:
    logger.error(e)

2021-11-29 05:51:30.926 | ERROR    | __main__:<module>:4 - Invalid count=-1, must > 0


In [6]:
count2colors(5)

('#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854')

In [7]:
count2colors(18)

('#1f77b4',
 '#aec7e8',
 '#ff7f0e',
 '#ffbb78',
 '#2ca02c',
 '#98df8a',
 '#d62728',
 '#ff9896',
 '#9467bd',
 '#c5b0d5',
 '#8c564b',
 '#c49c94',
 '#e377c2',
 '#f7b6d2',
 '#7f7f7f',
 '#c7c7c7',
 '#bcbd22',
 '#dbdb8d')

In [8]:
try:
    count2colors(1000)
except ValueError as e:
    logger.error(e)

2021-11-29 05:51:31.142 | ERROR    | __main__:<module>:4 - Invalid count=1000, must <= 768


In [9]:
#export
def bar_figure(
    data: dict, title="统计", with_label=True, y_log_multiple_thresh=1000, eps=1e-1, hide_legend=False,
    toolbar_location=BOKEH_DEFAULT_TOOLBAR_LOC, legend_orientation = "vertical", legend_location = "top_right", vertical_bar=True, 
    xaxis_major_label_orientation=math.pi/8, label_angle=0, bar_width=0.8, draw_line_circle=False, 
    tools=BOKEH_DEFAULT_TOOLS, x_axis_label=None, y_axis_label=None, **kwargs):
    x = [str(o) for o in data.keys()]
    y = list(data.values())
    
    source = ColumnDataSource(data=dict(x=x, y=y, color=count2colors(len(x))))
    if vertical_bar:
        p = figure(
            x_range=x, title=title,
            y_axis_type="log" if max(y)/(min(y)+eps) > y_log_multiple_thresh else 'linear',
            toolbar_location=toolbar_location, tools=tools, **kwargs)
        p.vbar(x='x', top='y', width=bar_width, bottom=0.1, color='color', legend_field="x", source=source)
    else:
        p = figure(
            y_range=x, title=title,
            toolbar_location=toolbar_location, tools=tools, **kwargs)
        p.hbar(y='x', right='y', height=bar_width, color='color', legend_field="x", source=source)
    p.legend.orientation = legend_orientation
    p.legend.location = legend_location
    if hide_legend: p.legend.visible=False 
    
    if with_label:
        if vertical_bar:
            labels = LabelSet(
                x='x', y='y', text='y', angle=label_angle,
                x_offset=-8, y_offset=2, source=source, render_mode='canvas', text_font_size="9pt")
            if draw_line_circle:
                p.line(x='x', y='y', source=source, line_color='#4292c6')
                p.circle(x='x', y='y', fill_color='white', size=6, source=source, line_color='black')
        else:
            labels = LabelSet(
                x='y', y='x', text='y', angle=label_angle,
                x_offset=2, y_offset=-8, source=source, render_mode='canvas', text_font_size="9pt")
        p.add_layout(labels)
    p.xgrid.grid_line_color = None
    p.xaxis.major_label_orientation = xaxis_major_label_orientation
    p.yaxis.axis_label = y_axis_label
    p.xaxis.axis_label = x_axis_label
    return p

In [10]:
output_notebook()

Loading BokehJS ...

In [11]:
test_data = {
    'A': 10000,
    'B': 1000,
    'C': 100,
    'D': 10
}
p = bar_figure(test_data, label_angle=math.pi/8, with_label=True)
show(p)

In [12]:
test_data = {
    'A': 100,
    'B': 88,
    'C': 99,
    'D': 10
}
p = bar_figure(test_data, vertical_bar=False)
show(p)

In [13]:
#export
def line_figure_datetime(
        data_list: list, plot_width=800, plot_height=500, x_dtype=np.datetime64,
        select_title:str=None, draw_circle=False, tooltips_metadata:list=None,
        x_name='x', y_name='y', legend_labels:list=None, tools=BOKEH_DEFAULT_TOOLS,
        use_select_figure=True, toolbar_location=BOKEH_DEFAULT_TOOLBAR_LOC, legend_alpha=0.8, 
        legend_orientation = "vertical", legend_location = "top_right",
        y_axis_label=None, x_axis_label=None, **kwargs
    ):
    if isinstance(data_list, dict): data_list = [data_list]
    if isinstance(tooltips_metadata, dict): tooltips_metadata = [tooltips_metadata]
    if not (isinstance(legend_labels, list) and len(legend_labels)) == len(data_list): legend_labels = None
    if not (isinstance(tooltips_metadata, list) and len(tooltips_metadata)) == len(data_list):
        tooltips_metadata = [{}]*len(data_list)
    p = figure(
        plot_height=int(0.65*plot_height), plot_width=plot_width,
        x_axis_type="datetime", x_axis_location="above" if use_select_figure else 'below',
        background_fill_color="#efefef", x_range=None,
        toolbar_location=toolbar_location,
        tools=tools, **kwargs
    )
    colors = count2colors(len(data_list))
    p.yaxis.axis_label = y_name if y_axis_label is None else y_axis_label
    p.xaxis.axis_label = x_name if x_axis_label is None else x_axis_label
    if use_select_figure:
        select_title = select_title if select_title is not None else f"拖动选择 {x_name} 区间"
        select = figure(
            title=select_title, tools=tools, toolbar_location=toolbar_location,
            plot_height=int(0.35*plot_height), plot_width=plot_width, y_range=p.y_range,
            x_axis_type="datetime", y_axis_type=None, background_fill_color="#efefef"
        )

    max_x = []
    for i, data in enumerate(data_list):
        if not isinstance(data, dict):
            logger.error(f"invalid data_list[{i}] data format, must be dict")
        x = [str(o) for o in data.keys()]
        if x_dtype: x = list(np.array(x, dtype=x_dtype))
        y = list(data.values())
        metadata = {}
        if isinstance(tooltips_metadata[i], dict):
            for k, v in tooltips_metadata[i].items():
                if isinstance(v, list) and len(v) == len(x):
                    metadata[k] = v
                else:
                    logger.warning(f"tooltips_metadata->{k} values invalid, type={type(v)} length={len(v)}")
        tooltips = [(x_name, "@{"f"{x_name}""}{%F}"),(y_name, "@{"f"{y_name}""}")]
        for k in metadata: tooltips.append((k, "@{"f"{k}""}"))
        column_data = {
            x_name: x,
            y_name: y
        }
        source = ColumnDataSource(data=dict(**column_data, **metadata))
        legend_kwargs = {} if legend_labels is None else {'legend_label': legend_labels[i]}
        p.line(x_name, y_name, source=source, line_color=colors[i], **legend_kwargs)
        if draw_circle: p.circle(x_name, y_name, fill_color='white', size=6, source=source, line_color=colors[i], **legend_kwargs)
        if len(x) > len(max_x): max_x = x
        p.add_tools(HoverTool(tooltips=tooltips, formatters={"@{"f"{x_name}""}": 'datetime'}))

        if use_select_figure: select.line(x_name, y_name, source=source, line_color=colors[i])
    p.x_scale = get_scale(p.x_range, 'datetime')
    p.legend.background_fill_alpha = legend_alpha
    p.legend.orientation = legend_orientation
    p.legend.location = legend_location
    if use_select_figure:
        p.x_range = get_range((max_x[len(max_x)//2-min(len(max_x)//8, 300)], max_x[len(max_x)//2+min(len(max_x)//8, 300)]))
        range_tool = RangeTool(x_range=p.x_range)
        range_tool.overlay.fill_color = "navy"
        range_tool.overlay.fill_alpha = 0.2

        select.ygrid.grid_line_color = None
        select.add_tools(range_tool)
        select.toolbar.active_multi = range_tool
        return column(p, select)
    else:
        return p

In [14]:
from bokeh.util.sampledata import _download_file
from pathlib import Path
import bokeh.sampledata
bokeh.sampledata.download()

Using data directory: /home/junxin/.bokeh/data
Skipping 'CGM.csv' (checksum match)
Skipping 'US_Counties.zip' (checksum match)
Skipping 'us_cities.json' (checksum match)
Skipping 'unemployment09.csv' (checksum match)
Skipping 'AAPL.csv' (checksum match)
Skipping 'FB.csv' (checksum match)
Skipping 'GOOG.csv' (checksum match)
Skipping 'IBM.csv' (checksum match)
Skipping 'MSFT.csv' (checksum match)
Skipping 'WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.zip' (checksum match)
Skipping 'gapminder_fertility.csv' (checksum match)
Skipping 'gapminder_population.csv' (checksum match)
Skipping 'gapminder_life_expectancy.csv' (checksum match)
Skipping 'gapminder_regions.csv' (checksum match)
Skipping 'world_cities.zip' (checksum match)
Skipping 'airports.json' (checksum match)
Skipping 'movies.db.zip' (checksum match)
Skipping 'airports.csv' (checksum match)
Skipping 'routes.csv' (checksum match)
Skipping 'haarcascade_frontalface_default.xml' (checksum match)


In [15]:
from bokeh.sampledata.stocks import AAPL

test_data = { x:y for x,y in zip(AAPL['date'][:300], AAPL['close'][:300])}
p = line_figure_datetime(test_data, title="趋势图", draw_circle=False, use_select_figure=False)
show(p)

/home/junxin/.local/lib/python3.8/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.background_fill_alpha` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/home/junxin/.local/lib/python3.8/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.orientation` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/home/junxin/.local/lib/python3.8/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be 

In [16]:
AAPL.keys()

dict_keys(['date', 'open', 'high', 'low', 'close', 'volume', 'adj_close'])

In [17]:
tooltips_metadata = {'体量': AAPL['volume'][:300]}
p = line_figure_datetime(test_data, title="趋势图", draw_circle=True,
                         tooltips_metadata=tooltips_metadata, x_name='时间',
                         y_name='价格')
show(p)

In [18]:
#export
def boxplot_figure(
    data_df, group_column='group', value_column='value', toolbar_location=BOKEH_DEFAULT_TOOLBAR_LOC,
    tools=BOKEH_DEFAULT_TOOLS, y_axis_label=None, x_axis_label=None, **kwargs):
    # find the quartiles and IQR for each category
    groups = df.groupby(group_column)
    q1 = groups.quantile(q=0.25)
    q2 = groups.quantile(q=0.5)
    q3 = groups.quantile(q=0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr
        # find the outliers for each category
    def outliers(group):
        cat = group.name
        return group[(group.score > upper.loc[cat][value_column]) | (group.score < lower.loc[cat][value_column])][value_column]
    out = groups.apply(outliers).dropna()

    # prepare outlier data for plotting, we need coordinates for every outlier.
    if not out.empty:
        outx = list(out.index.get_level_values(0))
        outy = list(out.values)

    p = figure(tools=tools, background_fill_color="#efefef", x_range=cats, toolbar_location=toolbar_location, **kwargs)

    # if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
    qmin = groups.quantile(q=0.00)
    qmax = groups.quantile(q=1.00)
    upper.score = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,value_column]),upper.score)]
    lower.score = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,value_column]),lower.score)]

    # stems
    p.segment(cats, upper.score, cats, q3.score, line_color="black")
    p.segment(cats, lower.score, cats, q1.score, line_color="black")

    # boxes
    p.vbar(cats, 0.7, q2.score, q3.score, fill_color="#E08E79", line_color="black")
    p.vbar(cats, 0.7, q1.score, q2.score, fill_color="#3B8686", line_color="black")

    # whiskers (almost-0 height rects simpler than segments)
    p.rect(cats, lower.score, 0.2, 0.01, line_color="black")
    p.rect(cats, upper.score, 0.2, 0.01, line_color="black")

    # outliers
    if not out.empty:
        p.circle(outx, outy, size=6, color="#F38630", fill_alpha=0.6)

    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = "white"
    p.grid.grid_line_width = 2
    p.xaxis.major_label_text_font_size="16px"
    p.yaxis.axis_label = y_axis_label
    p.xaxis.axis_label = x_axis_label
    return p

In [19]:
# generate some synthetic time series for six different categories
cats = list("abcdef")
yy = np.random.randn(2000)
g = np.random.choice(cats, 2000)
for i, l in enumerate(cats):
    yy[g == l] += i // 2
df = pd.DataFrame(dict(score=yy, group=g))
df.head()

,score,group
0,1.056522,e
1,2.871342,e
2,0.398220,b
3,-1.624127,a
4,1.015604,e


In [20]:
p = boxplot_figure(df, group_column='group', value_column='score')
show(p)

In [21]:
#export
def histogram_figure(
       hist, edges, title='histogram', x_axis_label='x', y_axis_label='y',
       fill_color='navy', toolbar_location=BOKEH_DEFAULT_TOOLBAR_LOC,
       tools=BOKEH_DEFAULT_TOOLS, **kwargs):
    p = figure(title=title, tools=tools, background_fill_color="#fafafa", toolbar_location=toolbar_location, **kwargs)
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color=fill_color, line_color="white", alpha=0.5)
    p.y_range.start = 0
    p.xaxis.axis_label = x_axis_label
    p.yaxis.axis_label = y_axis_label
    p.grid.grid_line_color="white"
    return p

In [22]:
# Normal Distribution

mu, sigma = 0, 0.5

measured = np.random.normal(mu, sigma, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

In [23]:
p = histogram_figure(hist, edges)
show(p)

In [24]:
#export
def scatter_figure(
        x, y, x_axis_label='x', y_axis_label='y', title='scatter',
        toolbar_location=BOKEH_DEFAULT_TOOLBAR_LOC, tools=BOKEH_DEFAULT_TOOLS, **kwargs
    ):
    assert len(x) == len(y)
    p = figure(title=title, tools=tools, toolbar_location=toolbar_location, **kwargs)
    p.xaxis.axis_label = x_axis_label
    p.yaxis.axis_label = y_axis_label

    p.circle(x, y, fill_alpha=0.2, size=10)
    return p

In [25]:
from bokeh.sampledata.iris import flowers
p = scatter_figure(flowers["petal_length"], flowers["petal_width"])
show(p)

In [26]:
#export
def bubble_figure(
    x, y, z=None, title='bubble', tools=BOKEH_DEFAULT_TOOLS, toolbar_location=BOKEH_DEFAULT_TOOLBAR_LOC,
    y_axis_label=None, x_axis_label=None, **kwargs):
    p = figure(tools=tools, title=title, toolbar_location=toolbar_location, **kwargs)
    if z is not None:
        z = np.array(z)
        z -= np.mean(z)
        z /= np.std(z)
        z /= ((max(z) - min(z))/2)
        z -= min(z)
        z /= 5
        z += 0.1
        p.scatter(x, y, fill_alpha=0.6, radius=z)
    else:
        p.scatter(x, y, fill_alpha=0.6)
    p.yaxis.axis_label = y_axis_label
    p.xaxis.axis_label = x_axis_label

    return p

In [27]:
flowers.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [28]:
p = bubble_figure(flowers["petal_length"], flowers["petal_width"], flowers["sepal_width"])
show(p)

In [29]:
#export
def stacked_bar_figure(
    data:dict, x_key, y_keys:list, title='stacked bar',
    toolbar_location=BOKEH_DEFAULT_TOOLBAR_LOC, tools=BOKEH_DEFAULT_TOOLS,
    y_axis_label=None, x_axis_label=None, **kwargs
):
    p = figure(title=title, toolbar_location=toolbar_location, tools=tools, x_range=data[x_key], **kwargs)

    p.vbar_stack(y_keys, x=x_key, width=0.9, color=colors, source=data, legend_label=y_keys)

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.axis.minor_tick_line_color = None
    p.outline_line_color = None
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    p.yaxis.axis_label = y_axis_label
    p.xaxis.axis_label = x_axis_label
    return p

In [30]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]
colors = ["#c9d9d3", "#718dbf", "#e84d60"]

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 4, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}
p = stacked_bar_figure(data, x_key='fruits', y_keys=['2015', '2016', '2017'])
show(p)

In [31]:
#export
def stacked_area_figure(
    df, stackers:list, title='stacked_area', tools=BOKEH_DEFAULT_TOOLS,
    toolbar_location=BOKEH_DEFAULT_TOOLBAR_LOC,
    y_axis_label=None, x_axis_label=None, **kwargs):
    p = figure(x_range=(0, len(df)-1), title=title, tools=tools, toolbar_location=toolbar_location, **kwargs)
    p.grid.minor_grid_line_color = '#eeeeee'
    if df.index.name is None: df.index.name = '_index_col'

    p.varea_stack(stackers=stackers, x=df.index.name, color=count2colors(len(stackers)), legend_label=stackers, source=df)

    # reverse the legend entries to match the stacked order
    p.legend.items.reverse()
    p.yaxis.axis_label = y_axis_label
    p.xaxis.axis_label = x_axis_label
    return p

In [32]:
df = pd.DataFrame(np.random.randint(10, 100, size=(15, 3))).add_prefix('y')
df.head()

,y0,y1,y2
0,97,88,73
1,43,92,62
2,73,79,59
3,61,72,44
4,46,96,36


In [33]:
p = stacked_area_figure(df, stackers=['y0', 'y1', 'y2'])
show(p)

In [34]:
#export
def pie_figure(data, title='pie', tools=BOKEH_DEFAULT_TOOLS, toolbar_location=BOKEH_DEFAULT_TOOLBAR_LOC,
                min_pct=0.01, others_name='OTHERS', legend_orientation="horizontal", legend_location="top_center",
                radius=0.8, pct_text_pad=100, **kwargs):
    x = list(data.keys())
    y = list(data.values())
    df = pd.Series(data).reset_index(name='y').rename(columns={'index':'x'})
    df["percentage"] = df["y"] / df["y"].sum()
    if isinstance(min_pct, float) and 0<min_pct<1 and len(df[df["percentage"] < min_pct]) > 0:
        others_y = df[df["percentage"] < min_pct]['y'].sum()
        others_pct = df[df["percentage"] < min_pct]['percentage'].sum()
        df = df[df["percentage"] >= min_pct]
        df = df.append({'x': others_name, 'y': others_y, 'percentage': others_pct}, ignore_index=True)
    df['percentage'] = df['percentage'].astype(float).map(lambda n: '{:.1%}'.format(n))
    df["percentage"] = df['percentage'].astype(str)
    df["percentage"] = df["percentage"].str.pad(pct_text_pad, side="left")
    df['angle'] = df['y']/df['y'].sum() * 2 * math.pi
    df['color'] = count2colors(len(df))
    tooltips = [("category", "@x"), ("count", "@y"), ("percentage", "@percentage")

]
    p = figure(title=title, tools=tools, tooltips=tooltips, toolbar_location=toolbar_location, **kwargs)

    p.wedge(x=0, y=1, radius=radius,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='x', source=df)
    # labels = LabelSet(x="cos", y="sin", y_offset=0, text='percentage', text_align="center", angle=0, source=ColumnDataSource(df), render_mode='canvas')
    labels = LabelSet(x=0, y=1, y_offset=0, text='percentage', text_align="center", angle=cumsum('angle', include_zero=True), source=ColumnDataSource(df), render_mode='canvas')
    p.add_layout(labels)

    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None
    p.legend.orientation = legend_orientation
    p.legend.location = legend_location
    return p

In [35]:
x = {
    'United States': 157,
    'United Kingdom': 93,
    'Japan': 89,
    'China': 63,
    'Germany': 44,
    'India': 42,
    'Italy': 40,
    'Australia': 35,
    'Brazil': 32,
    'France': 31,
    'Taiwan': 31,
    'Spain': 29
}
p = pie_figure(x)
show(p)

In [36]:
#export
def datatable_from_dataframe(df, **kwargs):
    df = df.fillna('nan')
    columns = [TableColumn(field=o, title=o) for o in df.columns]
    data_table = DataTable(columns=columns, source=ColumnDataSource(df), **kwargs)
    return data_table


In [37]:
show(datatable_from_dataframe(df))

In [38]:
#export
def bar_mixed_figure(df, group_keys:list, y_column_name, label_angle=0,
                    agg_method='sum', tools=BOKEH_DEFAULT_TOOLS, title='mixed bar',
                    toolbar_location=BOKEH_DEFAULT_TOOLBAR_LOC, draw_mean_line=False, line_alpha=0.8,
                    legend_orientation = "vertical", legend_location = "top_center",
                    label_offset_kwargs={'x_offset':-8, 'y_offset':2}, y_axis_label=None, x_axis_label=None,
                    line_legend_label="平均值", line_color="#4292c6", hide_legend=True, **kwargs):
    bar_group_keys = list(df[group_keys[-1]].unique())
    palette = count2colors(len(bar_group_keys))
    grouped_df = df.groupby(group_keys).agg(agg_method)
    bar_legend_values = list(grouped_df.reset_index()[group_keys[-1]].values)
    x = list(grouped_df.index)
    counts = list(grouped_df[y_column_name].values)

    source = ColumnDataSource(data=dict(x=x, counts=counts, bar_legend=bar_legend_values))

    p = figure(x_range=FactorRange(*x), title=title, toolbar_location=toolbar_location, tools=tools, **kwargs)

    p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white", legend_field='bar_legend',
        fill_color=factor_cmap('x', palette=palette, factors=bar_group_keys, start=len(group_keys)-1, end=len(group_keys)))
    labels = LabelSet(
                x='x', y='counts', text='counts', angle=label_angle,
                source=source, render_mode='canvas', text_font_size="9pt", **label_offset_kwargs)
    p.add_layout(labels)
    if draw_mean_line:
        line_data = df.groupby(group_keys[0])[y_column_name].mean().to_dict()
        p.line(
            x=list(line_data.keys()), y=list(line_data.values()), 
            color=line_color, line_width=2, legend_label=line_legend_label, alpha=line_alpha
        )
        p.circle(
            x=list(line_data.keys()), y=list(line_data.values()), 
            fill_color='white', size=6, line_color='black', legend_label=line_legend_label, alpha=line_alpha
        )
        p.add_tools(HoverTool(tooltips=[('mean', '@y')]))
    p.legend.orientation = legend_orientation
    p.legend.location = legend_location
    if hide_legend: p.legend.visible = False

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None
    p.yaxis.axis_label = y_axis_label
    p.xaxis.axis_label = x_axis_label
    return p

In [39]:
fruits_data_list = []
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
data = {
    '2019'   : [2, 1, 4, 3, 2, 4],
    '2020'   : [5, 3, 3, 2, 4, 6],
    '2021'   : [3, 2, 4, 4, 5, 3]
}
for year in data:
    for fruit, val in zip(fruits, data[year]):
        fruits_data_list.append([fruit, year, val])
fruits_df = pd.DataFrame(fruits_data_list, columns=['fruit', 'year', 'count'])

In [40]:
p = bar_mixed_figure(fruits_df[:-2], ['fruit', 'year'], 'count', draw_mean_line=True)
show(p)

In [41]:
#export
def text_figure(text, plot_width=600, plot_height=200, **kwargs):
    p = Paragraph(text=text, width=plot_width, height=plot_height, **kwargs)
    return p


In [42]:
text="""Your text is initialized with the 'text' argument.  The
remaining Paragraph arguments are 'width' and 'height'. For this example, those values
are 200 and 100, respectively."""
show(text_figure(text, plot_width=800))